In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [8]:
stat_pop1 = np.array (0)
for i in range (23):
    stat_pop1 = np.append (stat_pop1, 0)
for i in range (10):
    stat_pop1 = np.append (stat_pop1, 1)

In [11]:
stat_pop2 = np.array (0)
for i in range (11):
    stat_pop2 = np.append (stat_pop2, 0)
for i in range (4):
    stat_pop2 = np.append (stat_pop2, 1)

In [16]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [18]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [19]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [20]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(stat_pop1, stat_pop2)

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]


In [34]:
p_v1 = proportions_diff_z_test(proportions_diff_z_stat_ind(stat_pop1, stat_pop2), 'greater')
print "p-value: %f" % p_v1

p-value: 0.372930


In [35]:
round (p_v1,4)

0.3729

In [36]:
p= proportions_diff_z_test(proportions_diff_z_stat_ind(stat_pop1, stat_pop2), 'less')
print "p-value: %f" % p

p-value: 0.627070


In [75]:
from sklearn import model_selection, linear_model, metrics

In [140]:
data = pd.read_csv ('banknotes.txt', sep = '\t')

In [141]:
data_x = data[["X1" , "X2","X3","X4","X5","X6"]]
data_y = data['real']
print data_x.shape
print data_y.shape
print data.shape

(200, 6)
(200,)
(200, 7)


In [142]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(data_x, data_y, 
                                                                                     test_size = 0.25, random_state = 1)

In [144]:
log_regressor1 = linear_model.LogisticRegression()
log_regressor2 = linear_model.LogisticRegression()
log_regressor3 = linear_model.LogisticRegression()

In [145]:
log_regressor1.fit (train_data[['X1', 'X2','X3']], train_labels)
log_regressor2.fit (train_data[['X4', 'X5','X6']], train_labels)
log_regressor3.fit (train_data, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [146]:
predicts1 = log_regressor1.predict (test_data[['X1', 'X2','X3']])
predicts2 = log_regressor2.predict (test_data[['X4', 'X5','X6']])
predicts3 = log_regressor3.predict (test_data)

In [147]:
print metrics.accuracy_score(test_labels, predicts1)
print metrics.accuracy_score(test_labels, predicts2)
print metrics.accuracy_score(test_labels, predicts3)

0.8
0.98
0.98


In [169]:
prob1 =np.array(0)
i = 0
for z in zip(predicts1,test_labels):
    if i == 0:
        if z[0] == z[1]:
            prob1 = np.array(0)
        else:
            prob1 = np.array(1)
    else:
        if z[0] == z[1]:
            prob1 = np.append (prob1,0)
        else:
            prob1 = np.append (prob1,1)
    i +=1

In [170]:
prob1.shape

(50,)

In [171]:
prob2 =np.array(0)
i = 0
for z in zip(predicts2,test_labels):
    if i == 0:
        if z[0] == z[1]:
            prob2 = np.array(0)
        else:
            prob2 = np.array(1)
    else:
        if z[0] == z[1]:
            prob2 = np.append (prob2,0)
        else:
            prob2 = np.append (prob2,1)
    i +=1

In [172]:
prob2.shape

(50,)

In [77]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [78]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [159]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(test_labels != predicts1,test_labels != predicts2))

p-value: 0.003297


In [173]:
n_g, v_g = proportions_diff_confint_rel(prob1, prob2)
print n_g, v_g

0.059945206279614305 0.3000547937203857


In [162]:
round (n_g, 4)

0.0599

In [112]:
import math

In [135]:
z = (541.4-525)/(100/sqrt(100))

In [136]:
print z

1.6399999999999977


In [118]:
import scipy
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [138]:
p_v =  1 - scipy.stats.norm.cdf(z)
print round (p_v,4)

0.0505


In [133]:
z2 = (541.5-525)/(100/sqrt(100))

In [139]:
p_v1 =  1 - scipy.stats.norm.cdf(z2)
print round (p_v1,4)

0.0495
